# Efficient Python (part II)
* Efficient combining, counting, and iterating
* Pandas operations

In [7]:
# %load_ext line_profiler

import numpy as np
import pandas as pd
import collections 
import itertools

In [2]:
# combining objects
names = ['Bulbasaur', 'Charmander', 'Squirtle']
hps = [45, 39, 44]

# dict representation
dict(zip(names, hps))

{'Bulbasaur': 45, 'Charmander': 39, 'Squirtle': 44}

In [3]:
# tuple representation
[*zip(names, hps)]

[('Bulbasaur', 45), ('Charmander', 39), ('Squirtle', 44)]

### itertools module
Functional tools for creating & using iterations. It has the following functions:
- **Infinite iterations**: count, cycle, repeat
- **Finite iterations**: accumulate, chain, zip_longest, etc
- **Combination generators**: product, permutations, combinations

In [14]:
%%timeit
# combinations loop - BAD
pokemon_types = ['Bug', 'Fire', 'Water', 'Grass', 'Ghost']
combinations = []

for x in pokemon_types:
    for y in pokemon_types:
        if x == y:
            continue
        if ((x, y) not in combinations) and ((y, x) not in combinations):
            combinations.append((x, y))

6.57 µs ± 71.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
%%timeit
# combinations loop - BETTER
itertools.combinations(pokemon_types, 2)

123 ns ± 0.463 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [17]:
pokemon = ['Geodude', 'Cubone', 'Lickitung', 'Persian', 'Diglett']

# Create a combination object with pairs of Pokémon
combos_obj = itertools.combinations(pokemon, 2)
print(type(combos_obj), '\n')

# Convert combos_obj to a list by unpacking
combos_2 = [*combos_obj]
print(combos_2, '\n')

# Collect all possible combinations of 4 Pokémon directly into a list
combos_4 = [*itertools.combinations(pokemon, 4)]
print(combos_4)

<class 'itertools.combinations'> 

[('Geodude', 'Cubone'), ('Geodude', 'Lickitung'), ('Geodude', 'Persian'), ('Geodude', 'Diglett'), ('Cubone', 'Lickitung'), ('Cubone', 'Persian'), ('Cubone', 'Diglett'), ('Lickitung', 'Persian'), ('Lickitung', 'Diglett'), ('Persian', 'Diglett')] 

[('Geodude', 'Cubone', 'Lickitung', 'Persian'), ('Geodude', 'Cubone', 'Lickitung', 'Diglett'), ('Geodude', 'Cubone', 'Persian', 'Diglett'), ('Geodude', 'Lickitung', 'Persian', 'Diglett'), ('Cubone', 'Lickitung', 'Persian', 'Diglett')]


In [18]:
ash_pokedex = ['Pikachu', 'Bulbasaur', 'Koffing', 'Spearow', 'Vulpix', 
               'Wigglytuff', 'Zubat', 'Rattata', 'Psyduck', 'Squirtle']
misty_pokedex = ['Krabby', 'Horsea', 'Slowbro', 'Tentacool', 'Vaporeon', 
                 'Magikarp', 'Poliwag', 'Starmie', 'Psyduck', 'Squirtle']

# Convert both lists to sets
ash_set = set(ash_pokedex)
misty_set = set(misty_pokedex)

# Find the Pokémon that exist in both sets
both = ash_set.intersection(misty_set)
print(both)

# Find the Pokémon that Ash has and Misty does not have
ash_only = ash_set.difference(misty_set)
print(ash_only)

# Find the Pokémon that are in only one set (not both)
unique_to_set = ash_set.symmetric_difference(misty_set)
print(unique_to_set)

{'Squirtle', 'Psyduck'}
{'Koffing', 'Rattata', 'Vulpix', 'Bulbasaur', 'Zubat', 'Wigglytuff', 'Pikachu', 'Spearow'}
{'Tentacool', 'Koffing', 'Krabby', 'Rattata', 'Wigglytuff', 'Slowbro', 'Poliwag', 'Vulpix', 'Spearow', 'Bulbasaur', 'Zubat', 'Magikarp', 'Pikachu', 'Horsea', 'Vaporeon', 'Starmie'}


### Eliminating loops


In [23]:
pokemon_stats = np.random.rand(10, 4)
pokemon_stats *= 100
pokemon_stats

array([[81.83007415, 94.2035647 , 80.29301738, 11.20512775],
       [17.24294799, 90.91374051, 27.94736037, 14.67327874],
       [60.54105018, 78.24848911, 54.36872904, 89.2033941 ],
       [54.22740749, 53.09776311,  4.2176785 , 71.36045105],
       [31.53967203, 53.14693028, 45.0210001 , 93.79117127],
       [43.41229174, 13.28288775, 21.66522715, 99.47715929],
       [37.63984106, 54.77928923, 11.13321216, 33.80140551],
       [98.71511391, 89.67792957, 22.24727427, 45.6132658 ],
       [78.11585556, 21.27480201, 15.34871521, 23.32901087],
       [ 4.55484749, 48.9136162 , 20.2434094 , 48.36386917]])

In [29]:
%%timeit -r2 -n10
# for loop
total = []
for row in pokemon_stats:
    total.append(sum(row))

26 µs ± 1.01 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [30]:
%%timeit -r2 -n10
# list comprehension
total_comp = [sum(row) for row in pokemon_stats]

27 µs ± 925 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [31]:
%%timeit -r2 -n10
total_map = [*map(sum, pokemon_stats)]

16.8 µs ± 790 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [34]:
%%timeit 
# for loop vs numpy
avgs = []

for row in pokemon_stats:
    avgs.append(np.mean(row))


52.7 µs ± 6.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [35]:
%%timeit 
avgs_np = pokemon_stats.mean(axis=1)

5.02 µs ± 45.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
